# Setup

## Imports

In [1]:
import sqlalchemy as sa


In [2]:
from core.db import Base, get_engine, get_session

## Import models so the Base object can find the metadata it needs to create
from domain.models.demo import models as demo_models


In [3]:
from core.config import notebook_settings

NB_LOG = notebook_settings.NB_LOG
NB_VERBOSE = notebook_settings.NB_VERBOSE

## Global vars/functions

In [4]:
## Create custom database engine
#  Re-use value of NB_LOG to determine engine echo state
engine = get_engine(connection="db/demo.sqlite", echo=NB_VERBOSE)


In [5]:
## Create metadata
try:
    Base.metadata.create_all(bind=engine)
except Exception as exc:
    raise Exception(f"Unhandled exception creating Base metadata. Details: {exc}")


2023-06-04 16:09:24,004 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-04 16:09:24,005 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2023-06-04 16:09:24,008 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-04 16:09:24,011 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2023-06-04 16:09:24,012 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-04 16:09:24,014 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2023-06-04 16:09:24,015 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-04 16:09:24,016 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2023-06-04 16:09:24,017 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-04 16:09:24,019 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30) NOT NULL, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2023-06-04 16:09:24,021 INFO sqlalchemy.engine.Engine [no key 0.00141s] ()
2023-06-04 16:09:24,035 INFO sqlalchemy.engine.

In [6]:
## Create custom database session
#  Use expire_on_commit=False for notebook sessions
SessionLocal = get_session(engine=engine, expire_on_commit=False)
print(f"SessionLocal: {type(SessionLocal)}")


SessionLocal: <class 'sqlalchemy.orm.session.sessionmaker'>


In [7]:
def demo_add_users():
    """
    Demo function, creates static users on the fly and
    commits them to the database.
    """

    try:
        with SessionLocal() as sess:
            ## User1, all fields with single address
            spongebob = demo_models.User(
                name="spongebob",
                fullname="Spongebob Squarepants",
                addresses=[
                    demo_models.Address(email_address="spongebob@sqlalchemy.org")
                ],
            )

            ## User 2, all fields with multiple addressess
            sandy = demo_models.User(
                name="sandy",
                fullname="Sandy Cheeks",
                addresses=[
                    demo_models.Address(email_address="sandy@sqlalchemy.org"),
                    demo_models.Address(email_address="sandy@squirrelpower.org"),
                ],
            )

            ## User 3, naame & fullname, no address
            patrick = demo_models.User(name="patrick", fullname="Patrick Star")

            ## Build list of users to pass to add_all()
            _users: list[demo_models.User] = [spongebob, sandy, patrick]
            add_users: list[demo_models.User] = []

            for _user in _users:
                ## Check if _user already exists
                if (
                    not sess.query(demo_models.User)
                    .filter(demo_models.User.name == _user.name)
                    .count()
                ):
                    ## _user does not exist, add to add_users list
                    add_users.append(_user)
                else:
                    print(f"User '{_user.name}' already exists.")
                    pass

            print(f"Add users: {add_users}")

            ## Add all users to session
            sess.add_all(add_users)

            ## Commit to database
            sess.commit()

    except Exception as exc:
        raise Exception(f"Unhandled exception starting session. Details: {exc}")

In [8]:
def demo_select(
    user_names: list[str] = ["spongebob", "sandy"]
) -> list[demo_models.User]:
    statement: sa.sql.Select = sa.select(demo_models.User).where(
        demo_models.User.name.in_(user_names)
    )

    with SessionLocal() as sess:
        all_users = sess.scalars(statement)

        users: list[demo_models.User] = all_users.all()

    return users


# Operations

In [9]:
add_users = demo_add_users()


2023-06-04 16:09:24,201 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-04 16:09:24,211 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.name = ?) AS anon_1
2023-06-04 16:09:24,213 INFO sqlalchemy.engine.Engine [generated in 0.00156s] ('spongebob',)
2023-06-04 16:09:24,216 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.name = ?) AS anon_1
2023-06-04 16:09:24,218 INFO sqlalchemy.engine.Engine [cached since 0.007098s ago] ('sandy',)
2023-06-04 16:09:24,221 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.full

In [10]:
select_users = demo_select()

2023-06-04 16:09:24,295 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-04 16:09:24,300 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name IN (?, ?)
2023-06-04 16:09:24,301 INFO sqlalchemy.engine.Engine [generated in 0.00115s] ('spongebob', 'sandy')
2023-06-04 16:09:24,306 INFO sqlalchemy.engine.Engine ROLLBACK


In [11]:
if NB_LOG:
    display(f"Users: {select_users}")


"Users: [User(id=1, name='spongebob', fullname='Spongebob Squarepants'), User(id=2, name='sandy', fullname='Sandy Cheeks')]"